In [1]:
import mmcv
from mmcv import Config
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)
from mmdet.models import build_detector
from mmdet.apis import single_gpu_test
from mmcv.runner import load_checkpoint
import os
from mmcv.parallel import MMDataParallel
import pandas as pd
from pandas import DataFrame
from pycocotools.coco import COCO
import numpy as np

/opt/conda/envs/detection/lib/python3.7/site-packages/mmcv/__init__.py:21: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  'On January 1, 2023, MMCV will release v2.0.0, in which it will remove '


In [8]:
classes = ("General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")

# config file 들고오기
cfg = Config.fromfile('/opt/ml/baseline/baseline_cv11/UniverseNet/configs/universenet/universenet101_2008d_fp16_4x4_mstrain_480_960_20e_coco.py')

root='../../../dataset/'

epoch = 'best_bbox_mAP_50_epoch_20'

# dataset config 수정
cfg.data.test.classes = classes
cfg.data.test.img_prefix = root
cfg.data.test.ann_file = root + 'test.json'
cfg.data.test.pipeline[1]['img_scale'] = (512,512) # Resize
cfg.data.test.test_mode = True

cfg.data.samples_per_gpu = 4

cfg.seed=2021
cfg.gpu_ids = [1]
cfg.work_dir = '/opt/ml/baseline/baseline_cv11/UniverseNet/work_dirs/universenet101_2008d_fp16_4x4_mstrain_480_960_20e_coco_trash'

if "roi_head" in cfg.model.keys():
        if type(cfg.model.roi_head.bbox_head) == dict:
            cfg.model.roi_head.bbox_head.num_classes = 10

        #In case of cascade RCNN : List[Dict]
        elif type(cfg.model.roi_head.bbox_head) == list:
            for each_head in cfg.model.roi_head.bbox_head:
                if hasattr(each_head, "num_classes"):
                    each_head.num_classes = 10 
                else: 
                    raise Exception("Num_classes가 없습니다")
else:
    cfg.model.bbox_head.num_classes = 10
cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)
cfg.model.train_cfg = None

In [9]:
# build dataset & dataloader
dataset = build_dataset(cfg.data.test)
data_loader = build_dataloader(
        dataset,
        samples_per_gpu=1,
        workers_per_gpu=cfg.data.workers_per_gpu,
        dist=False,
        shuffle=False)

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


In [10]:
# checkpoint path
checkpoint_path = os.path.join(cfg.work_dir, f'{epoch}.pth')

model = build_detector(cfg.model, test_cfg=cfg.get('test_cfg')) # build detector
checkpoint = load_checkpoint(model, checkpoint_path, map_location='cpu') # ckpt load

model.CLASSES = dataset.CLASSES
model = MMDataParallel(model.cuda(), device_ids=[0])

load checkpoint from local path: /opt/ml/baseline/baseline_cv11/UniverseNet/work_dirs/universenet101_2008d_fp16_4x4_mstrain_480_960_20e_coco_trash/best_bbox_mAP_50_epoch_20.pth


2023-05-10 12:51:39,698 - root - INFO - DeformConv2dPack backbone.layer2.0.convs.0 is upgraded to version 2.
2023-05-10 12:51:39,700 - root - INFO - DeformConv2dPack backbone.layer2.0.convs.1 is upgraded to version 2.
2023-05-10 12:51:39,701 - root - INFO - DeformConv2dPack backbone.layer2.0.convs.2 is upgraded to version 2.
2023-05-10 12:51:39,710 - root - INFO - DeformConv2dPack backbone.layer2.1.convs.0 is upgraded to version 2.
2023-05-10 12:51:39,712 - root - INFO - DeformConv2dPack backbone.layer2.1.convs.1 is upgraded to version 2.
2023-05-10 12:51:39,713 - root - INFO - DeformConv2dPack backbone.layer2.1.convs.2 is upgraded to version 2.
2023-05-10 12:51:39,722 - root - INFO - DeformConv2dPack backbone.layer2.2.convs.0 is upgraded to version 2.
2023-05-10 12:51:39,723 - root - INFO - DeformConv2dPack backbone.layer2.2.convs.1 is upgraded to version 2.
2023-05-10 12:51:39,724 - root - INFO - DeformConv2dPack backbone.layer2.2.convs.2 is upgraded to version 2.
2023-05-10 12:51:39

In [11]:
output = single_gpu_test(model, data_loader, show_score_thr=0.05) # output 계산

[>>>>>>>>>>>>>>>>>>>>>>>>>>>] 4871/4871, 7.3 task/s, elapsed: 667s, ETA:     0s

In [12]:
# submission 양식에 맞게 output 후처리
prediction_strings = []
file_names = []
coco = COCO(cfg.data.test.ann_file)
img_ids = coco.getImgIds()

class_num = 10
for i, out in enumerate(output):
    prediction_string = ''
    image_info = coco.loadImgs(coco.getImgIds(imgIds=i))[0]
    for j in range(class_num):
        for o in out[j]:
            prediction_string += str(j) + ' ' + str(o[4]) + ' ' + str(o[0]) + ' ' + str(o[1]) + ' ' + str(
                o[2]) + ' ' + str(o[3]) + ' '
        
    prediction_strings.append(prediction_string)
    file_names.append(image_info['file_name'])


submission = pd.DataFrame()
submission['PredictionString'] = prediction_strings
submission['image_id'] = file_names
submission.to_csv(os.path.join(cfg.work_dir, f'submission_{epoch}.csv'), index=None)
submission.head()

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


,PredictionString,image_id
0,0 0.23994805 556.54785 350.56863 708.01605 607...,test/0000.jpg
1,0 0.21525708 636.2284 655.3315 689.5049 844.98...,test/0001.jpg
2,0 0.6501753 778.7716 389.85974 1023.49835 1024...,test/0002.jpg
3,0 0.4281529 0.059143066 0.0 1023.682 1009.3923...,test/0003.jpg
4,0 0.8571342 185.49188 255.19092 878.39685 777....,test/0004.jpg
